<h1 dir="rtl" align=center>
تخمین قیمت گوشی های تلفن همراه براساس آگهی دیوار
</h1>
<h2 dir="rtl" align=center>
محمدرضا عظیمی
</h2>
<h3 dir="rtl" align=center>
۸۱۰۱۹۶۵۱۰
</h3>

<div lang="fa" dir="rtl">
<h2>
کلاس Regressor
</h2>
<p>
میخواهیم با استفاده روش های هوش مصنوعی، تخمین قیمت تلفن های همراه را انجام دهیم. برای این کار کلاس Regressor
را طراحی میکنیم.

این کلاس داده های مورد نظر را از دیتا ست خوانده و با پیش پردازش و تمیز کردن آن ها و سپس استخراج ویژگی های مختلف، به یادگیری از روی داده ها می پردازد.
</p>

<h2>
کتابخانه scikit learn
</h2>
<p>
در این پروژه از کتابخانه scikit learn
برای پردازش داده ها و یادگیری استفاده می کنیم.

این کتابخانه توابع مناسبی جهت جدا کردن، پیش پردازش و یادگیری رگرسیون دارد که در ادامه از آن ها استفاده می کنیم.
</p>
</div>

In [ ]:
!pip install hazm
!pip install unidecode

from __future__ import unicode_literals
import io
import sys
import pandas as pd
import re
import numpy
import operator
import random
from hazm import *
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
from unidecode import unidecode

class Regressor:
	def __init__(self):
		self.train = list()
		self.evaluation = list()
		self.X_train = list()
		self.X_test = list()
		self.y_train = list()
		self.y_test = list()
		self.reg = LinearRegression()

<div lang="fa" dir="rtl">
<h2>
پیش پردازش
</h2>

<p>
تابع clean
که یک تابع static
می باشد، یک جمله را بعنوان ورودی گرفته و عملیات تمیز کردن داده را روی آن انجام می دهد.

در این تابع که از بسته ی فارسی هضم استفاده می کند، ابتدا جملات را نرمال سازی کرده و حروف بی ربط و اضافی را از آن پاک می کنیم.

سپس کلمات هر جمله را جدا کرده و با استفاده از تابع lemmatization
کلمه مورد نظر را به ریشه ی اصلی اش بازمی گردانیم.

این کار باعث می شود تا از شلوغی داده ها کاسته شده و راحت تر بتوانیم داده ها را پردازش کنیم.
</p>
<h3>
پیدا کردن ریشه ی کلمات
</h3>

<p>

برای این کار می توانیم از دو روش stemming
و یا lemmatization
استفاده کنیم.
<br>
در روش stemming،
به معنی کلمات توجه نمیشود و فقط بخش پایانی کلمات مانند 'ها' در 'آن ها' و پسوند های کلمات، صفات و افعال حذف میشوند.
همچنین در این روش به اشکال مختلف افعال و زمان آن ها، توجه نمیشود. بنابراین این روش به خوبی ریشه ی کلمات را به دست نمیدهد.

اما این روش سریع است

اما در روش lemmatization به معنی کلمات توجه میشود و تنها درصورتی پسوند کلمات حذف میشوند که کلمه ی جدید با معنی باشد. در نتیجه کلماتی که از این روش بدست می آیند دارای معنی بوده و ریشه ی واقعی کلمه تولید میشود.
البته ممکن است ریشه ی بعضی از کلمات را نتواند بدست بیاورد و خود کلمه رو برگرداند.

از آنجاییکه متن شامل غلط های املایی بسیاری است، استفاده از lemmatization
کمک زیادی به ما نمی کند و نمی تواند ریشه ی کلمات را بدست دهد.

از این رو از stemmer استفاده میکنیم.
</p>
</div>

In [ ]:
	@staticmethod
	def clean(sentence):
		normalizer = Normalizer()
		sentence = normalizer.normalize(sentence)
		words = word_tokenize(sentence)
		new_words = []

		for word in words:
			new_words.append(Stemmer().stem(word))

		return new_words

<div lang="fa" dir="rtl">
<h3>
ویژگی ها در مجموعه دادگان
</h3>
<p>
در دیتاست موجود، ۷ ویژگی وجود دارد که ویژگی هشتم قیمت بوده و ما باید آن را تخمین بزنیم.

۱. برند

این ویژگی تاثیر مهمی در قیمت موبایل دارد و مقادیر این ستون محدود می باشند.

بنابراین از روش label encoding
استفاده کرده و به هر برند یک عدد اختصاص می دهیم.

این روش در این پروژه از روش one hot encoding
بهتر است زیرا برند ها باهم قابل مقایسه بوده و برخی از دیگری گران ترند.

برای مثال برند اپل معمولا قیمت های بالایی دارد.

۲. شهر

شهر محل فروش تاثیر بسزایی در قیمت یک موبایل ندارد، اما می توان گفت که در شهرهای کم جمعیت ممکن است موبایل با قیمت متفاوتی نسبت به شهرهای پرجمعیت به فروش برسد.

بنابراین برای ویژگی شهر نیز دقیقا مانند برند عمل کرده و به هر شهر عددی اختصاص می دهیم.

۳. عنوان

عنوان یک آگهی اطلاعات مفیدی را به ما می دهید.

از جمله مهم ترین اطلاعاتی که می شود از عنوان برداشت کرد، مدل گوشی است.

معمولا مدل موبایل در عنوان آگهی ذکر می شود که تاثیر بسزایی در قیمت آن موبایل دارد.

برای مثال بین مدل های مختلف برند اپل تفاوت قیمت وجود دارد.

همچنین در عنوان آگهی می توان اطلاعاتی درمورد ویژگی های سخت افزاری از جمله رم موبایل تیز بدست آورد.

۴. توضیحات

توضیحات نیز می تواند اطلاعاتی درمورد ویژگی های جزئی تر موبایل به ما بدهد.

در این ستون علاوه بر اطلاعات موجود در ستون عنوان، اطلاعاتی مانند نو یا کهنه بودن، شکستگی، سلامت، گرانتی، اقلام همراه، جعبه، خط و خش و سایر ایرادها نیز بدست آورد.

با پردازش این متن توضیحات میتوان اطلاعات مفیدی را بدست آورد.

۵. تعداد تصاویر

تعداد تصاویری که از یک کالا موجود است می تواند بر سالم و تمیز بودن کالا دلالت داشته باشد و در نتیجه در قیمت موثر باشد.

۶. زمان ایجاد آگهی

از آنجایی که زمان آگهی فقط روز هفته و ساعت را اعلام کرده، نمیتواند تاثیر زیادی بر قیمت موبایل داشته باشد.

این ویژگی واریانس پایینی دارد و از آن صرف نظر می کنیم.

۷. قیمت

این ویژگی عددی غیر اعشاری است که ما باید آن را تخمین بزنیم.

در برخی از سطرها قیمت موجود نیست و این سطرها باید از داده ها حذف شوند.

همچنین در برخی از سطرها نیز چند صفر از جلوی قیمت برداشته شده است.

این موارد نقص های دیتاست بوده و می بایست آن ها را برطرف و یا از مجموعه دادگان حذف کرد.
</p>
</div>

In [ ]:
	@staticmethod
	def extract_ram_model(sentence, brands):
		brands = [b.lower() for b in brands]
		brands.extend(['iphone', 'asus', 'galaxy', 'lumia', 'honor'])
		english = [ch for ch in sentence if (ord(ch) <= ord('9') and ord(ch) >= ord('0')) or ch is ' ' \
            or (ord(ch) <= ord('z') and ord(ch) >= ord('a')) or (ord(ch) <= ord('Z') and ord(ch) >= ord('A'))]
		english = ''.join(english).lower().split()
		english = [w for w in english if w not in brands]
		# print(english)

		ram = 0
		flag = False
		has_ram = False
		for word in english:

			for brand in brands:
				if brand in word:
					flag = True

			if '16' in word:
				has_ram = True
				ram = 16
			elif '32' in word:
				has_ram = True
				ram = 32
			elif '64' in word:
				has_ram = True
				ram = 64
			elif '128' in word:
				has_ram = True
				ram = 128
			elif '256' in word:
				has_ram = True
				ram = 256

			elif 'g' in word.lower():
				has_ram = True
				if '4' in word:
					ram = 4
				elif '8' in word:
					ram = 8

			elif word.lower() == 'gig' or word.lower() == 'gb':
				has_ram = True
				ram = max([int(num) for num in english if num.isdigit()])

			if has_ram:
				english.remove(word)

		if not has_ram:
			ram = 0
			flag = False
			has_ram = False
			for word in unidecode(''.join(english)):

				for brand in brands:
					if brand in word:
						flag = True

				if '16' in word:
					has_ram = True
					ram = 16
				elif '32' in word:
					has_ram = True
					ram = 32
				elif '64' in word:
					has_ram = True
					ram = 64
				elif '128' in word:
					has_ram = True
					ram = 128
				elif '256' in word:
					has_ram = True
					ram = 256

				elif 'گیگ' in word.lower():
					has_ram = True
					if '4' in word:
						ram = 4
					elif '8' in word:
						ram = 8

				elif word.lower() == 'گیگ' or word.lower() == 'گیگابایت':
					has_ram = True
					ram = max([int(num) for num in english if num.isdigit()])

				if has_ram:
					english.remove(word)

		model = 'unknown'
		if (flag and len(english) > 0):
			ok = False
			for br in brands:
				if br in english[0]:
					ok = True

			if ok:
				model = english[0]
				# print(model)

		return [ram, model]

<div lang="fa" dir="rtl">
<h3>
استخراج ویژگی ها
</h3>
<p>
درمورد ویژگی هایی مانند برند و شهر پیش تر صحبت شد و استخراج آن ها بسادگی و با روش label encoding قابل انجام بود.

اما برخی از ویژگی ها درون متن ها نهفته اند و باید آن ها را به دقت استخراج کنیم:

۱. رم

حجم رم یک موبایل تاثیر بسزایی در قیمت آن دارد.

این اطلاعات در عنوان و یا توضیحات ذکر می شوند و می توان آن ها را استخراج کرد.

برای این کار دنبال کلمات کلیدی مانند گیگ، gb، gig، ‌‌g و غیره 
می گردیم.

درصورت یافتن این کلمات در متن، می توانیم حجم رم را در اطراف این کلمات پیدا کنیم.

همچنین برخی از اعداد بصورت فارسی نوشته شده اند که با استفاده از تابع unidecode
آن ها را به انگلیسی تبدیل می کنیم.

۲. امتیاز

یک ویژگی جدید بنام امتیاز تعریف می کنیم که با بر اساس تعداد کلمات مثبت و منفی موجودد در توضیحات آگهی تعیین می شود.

برای مثال آگهی ای که دارای کلمات سالم بدون خط و .. است امتیاز مثبت و آگهی ای که شامل کلماتی از قبیل شکسته، ترک و...
است امتیاز منفی می گیرد.

۳. مدل گوشی

مورد دیگری که در توضیحات پنهان است مدل گوشی می باشد.

سعی کردیم تا مدل های مختلفی را بر اساس برند موبایل پیدا کرده و به هر کدام برچسبی بدهیم و آن را به یک ویژگی تبدیل کنیم.

در ادامه تابع پیش پردازش را مشاهده می کنیم.
</p>
<h3>
scale کردن ویژگی ها
</h3>
<p>
برای بهتر شدن دقت و استاندارد شدن داده ها، آن ها را بر اساس میانگین و واریانس نرمال کرده و سپس از آن ها استفاده می کنیم.

این کار را با استفاده از تابع StandardScaler
موجود در بسته ی scikit learn
انجام می دهیم.
</p>

<h3>
جدا کردن داده های تمرین و تست
</h3>
<p>
با استفاده از تابع train_test_split، ۸۰ درصد
از داده ها را برای یادگیری و ۲۰ درصد را برای تست جدا می کنیم.

این کار را بصورت تصادفی و توسط همین تابع انجام می دهیم.
</p>
</div>

In [ ]:
	def pre_process(self, data_file):
		df = pd.read_csv(data_file)
		data = df.loc[(df['price'] > 1000)]
		cities = set(data['city'])
		brands = set(data['brand'])
		brands = set([b.split(':')[0] for b in brands])
		city_dict = {city : 0 for city in cities}
		brand_dict = {brand : 0 for brand in brands}

		i = 0
		for k, _ in city_dict.items():
			city_dict[k] = i
			i += 1

		i = 0
		for k, _ in brand_dict.items():
			brand_dict[k] = i
			i += 1

		data.insert(2, 'ram', np.zeros(len(data)), True)
		data.insert(2, 'rate', np.zeros(len(data)), True)
		data.insert(2, 'model', ['' for i in range(len(data))], True)
		for i, _ in data.iterrows():
			data.at[i, 'city'] = city_dict[data['city'][i]]
			data.at[i, 'brand'] = brand_dict[data['brand'][i].split(':')[0]]
			title = data.at[i, 'title']
			desc = data.at[i, 'desc']
			t_d = title + desc
			ram_model = self.extract_ram_model(t_d, brands)
			data.at[i, 'ram'] = ram_model[0]
			data.at[i, 'model'] = ram_model[1]
			title = self.clean(title)
			info = title.copy()
			info.extend(self.clean(desc))
			goods = ['سالم', 'نو', 'اک', 'آک', 'درحد', 'تمیز', 'بسیار', 'کیف', 'هنذفری', 'پلمپ', 'بدون', 'سالمه', 'فابریک', 'شارژر',\
                    'تعمیرنرفته', 'کاملا', 'چرم', 'درحداک', 'کارتون', 'کارتن', 'تست', 'گارانتی', 'اورجینال', 'خط']
			bads = ['شکسته', 'ایراد', 'ایرادش', 'شکست', 'تاچ', 'افتاده', 'ترک', 'خوردگ', 'خوردگی', 'افتاد']

			rate = 0.0
			for word in info:
				if word in goods:
					rate += 1
				elif word in bads:
					rate -= 1

			data.at[i, 'rate'] = rate

		models = set(data['model'])
		model_dict = {model : 0 for model in models}
		i = 0
		for k, _ in model_dict.items():
			model_dict[k] = i
			i += 1

		for i, _ in data.iterrows():
			data.at[i, 'model'] = model_dict[data['model'][i]]

		scaler = StandardScaler()
		scaler.fit(data[['brand', 'city', 'image_count', 'ram', 'rate', 'model']])
		scaled_data = scaler.transform(data[['brand', 'city', 'image_count', 'ram', 'rate', 'model']])

		self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(scaled_data, data['price'], test_size=0.2, random_state=42)

<div lang="fa" dir="rtl">
<h2>
یادگیری و تست
</h2>
<p>
برای یادگیری از LinearRegression
استفاده می کنیم.

تابع fit این کلاس ویژگی
های ما را گرفته و مدلی آموزش دیده شده را تولید می کند.

همچنین با استفاده از تابع predict
قیمت را برای داده های تست خود نیز محاسبه کرده و در نهایت خطا را بصورت مجموع قدر مطلق خطاها بدست آورده و نتیجه کار خود را مشاهده میکنیم:
</p>
</div>

In [ ]:
	def learn(self):
		self.reg.fit(self.X_train, self.y_train)
		y_pred = self.reg.predict(self.X_test)

		loss = 0
		for x, y in zip(y_pred, self.y_test):
			loss += abs(x - y)

		print('Loss:', loss / len(self.y_test))

	X_train : list
	X_test : list
	y_train : list
	y_test : list
	reg : LinearRegression

<div lang="fa" dir="rtl">
<h2>
مشاهده نتیجه
</h2>
<p>
اکنون شی ای از کلاس Regressor
ساخته و توابع را فراخوانی میکنیم تا مقدار خطا را مشاهده کنیم.
</p>
</div>

In [ ]:
def main():
	regressor = Regressor()
	regressor.pre_process(open("mobile_phone_dataset.csv", newline=''))
	regressor.learn()

if __name__ == "__main__":
	main()

Loss: 354344.894938


<div lang="fa" dir="rtl">
<h2>
تحلیل و بررسی
</h2>
<p>
همانطور که مشاهده می کنیم، تخمینگر ما حدودا ۳۵۰ هزار تومان خطا داشته که خیلی دقیق نیست اما تقریب خوبی از قیمت به ما می دهد.

از دلایل عدم موفقیت کامل می توان به موارد زیر اشاره کرد:

۱. دیتاست نامرتب

۲. ترکیب کلمات و حروف فارسی و انگلیسی

۳. عدم بیان ویژگی های مشخص در برخی آگهی ها

۴. نقص های داده ای مانند قیمت های پرت
</p>

<h2>
کارهای قابل انجام
</h2>
<p>
زمان در این پروژه محدود بود اما در آینده می توان برخی اعمال را جهت بهتر کردن دقت انجام داد.

این اعمال عبارتند از:

۱. حذف یا اصلاح برخی قیمت های پرت

برای مثال قیمت های خیلی پایین را حذف کرده و یا در ۱۰۰۰ ضرب کنیم.

۲. بررسی سایر مشخصات سخت افزاری

در اینجا ما فقط رم را بررسی کردیم درحالیکه میتوان سایر موارد مانند حافظه جانبی، دوربین و .. را نیز از توضیحات بیرون کشید و آن ها را بررسی کرد.

۳. استخراج دقیقتر مدل گوشی

مدل یک گوشی تعیین کننده ترین معیار برای قیمت آن پس از برند است.

از آنجاییکه این مورد به راحتی از دادگان قابل استخراج نیست، می توان با دقت بیشتری و یا جمع آوری تمامی مدل های ممکن از گوشی، مدل را نیز بصورت دقیق تر استخراج کرد.

۴. استفاده از روش های پردازش متن

برخی روش ها مانند tf-idf
می توانند ما را در تشخیص کلمات مهم و تاثیر گذار در قبمت یاری کرده و دقت مدل را بهبود بخشند.
</p>
</div>